In [ ]:
import pandas as pd

In [ ]:
latest_df = pd.read_csv(
    "../../reports/deidentified_overdose_201201202408_zips_0311.csv"
)
drug_cols = [
    "Methamphetamine",
    "Heroin",
    "Cocaine",
    "Fentanyl",
    "Alcohol",
    "Prescription.opioids",
    "Any Opioids",
    "Benzodiazepines",
    "Others",
    "Any Drugs",
]

In [ ]:
latest_df.columns

In [ ]:
# 2. Convert 'DeathDate' to a Date/Time type (if it's not already)
latest_df["DeathDate"] = pd.to_datetime(latest_df["DeathDate"], errors="coerce")

# (Optional) If your dataset uses 'MonthYear' instead, parse that:
# latest_df["MonthYear"] = pd.to_datetime(latest_df["MonthYear"], format="%Y-%m")

# 3. Filter if you only want "Any Drugs" overdoses (i.e., rows where 'Any Drugs' == 1)
df_any = latest_df[latest_df["Any Drugs"] == 1].copy()

# 4. Create a monthly or quarterly period column
#    For monthly:
df_any["YearMonth"] = df_any["DeathDate"].dt.to_period("M")

#    OR for quarterly:
# df_any["YearQuarter"] = df_any["DeathDate"].dt.to_period("Q")

# 5. Group by ZIP code and month to count overdoses
time_col = "YearMonth"  # or "YearQuarter"
df_counts = (
    df_any.groupby(["ZIPCODE", time_col])
    .size()  # counts the number of rows
    .reset_index(name="Overdose_Count")
)

# 6. Create an "All" row by aggregating across all ZIP codes
df_all = df_any.groupby(time_col).size().reset_index(name="Overdose_Count")
df_all["ZIPCODE"] = "All"

# Combine them
df_counts = pd.concat([df_counts, df_all], ignore_index=True)

# 7. (Optional) Sort by time so we can compute rolling averages
df_counts = df_counts.sort_values(["ZIPCODE", time_col])

# Convert the period column to a datetime start-of-period for rolling calculations
df_counts["PeriodStart"] = df_counts[time_col].dt.to_timestamp(how="start")

# 8. (Optional) Compute a rolling 12-month average (for monthly data)
# For quarterly data, rolling(4) approximates a 12-month rolling average
df_counts["Rolling12"] = df_counts.groupby("ZIPCODE")["Overdose_Count"].transform(
    lambda x: x.rolling(window=12, min_periods=1).mean()
)

# 9. Save to CSV for ArcGIS Online
df_counts.to_csv("overdose_monthly_timeseries.csv", index=False)

In [ ]:
df_counts

### Time Series with all 

In [ ]:
import pandas as pd

# 1. Load your raw overdose data (one row per overdose case)
latest_df = pd.read_csv(
    "../../reports/deidentified_overdose_201201202408_zips_0311.csv"
)

drug_cols = [
    "Methamphetamine",
    "Heroin",
    "Cocaine",
    "Fentanyl",
    "Alcohol",
    "Prescription.opioids",
    "Any Opioids",
    "Benzodiazepines",
    "Others",
    "Any Drugs",
]

# Melt so each row is (Case, Overdose_Type)
df_melt = latest_df.melt(
    id_vars=["CaseNumber", "ZIPCODE", "DeathDate"],  # plus any other fields you need
    value_vars=drug_cols,
    var_name="Overdose_Type",
    value_name="Occurred",
)

# Filter only rows where Occurred == 1 (drug was involved)
df_melt = df_melt[df_melt["Occurred"] == 1]
df_melt.drop(columns=["Occurred"], inplace=True)

In [ ]:
# Convert 'DeathDate' to datetime if not already
df_melt["DeathDate"] = pd.to_datetime(df_melt["DeathDate"], errors="coerce")

# Create a monthly or quarterly period
df_melt["YearMonth"] = df_melt["DeathDate"].dt.to_period("M")
# If you want quarters: df_melt["YearQuarter"] = df_melt["DeathDate"].dt.to_period("Q")
time_col = "YearMonth"  # or "YearQuarter"

In [ ]:
df_counts = (
    df_melt.groupby(["ZIPCODE", "Overdose_Type", time_col])
    .size()
    .reset_index(name="Overdose_Count")
)

In [ ]:
df_all_zip = (
    df_melt.groupby(["Overdose_Type", time_col])
    .size()
    .reset_index(name="Overdose_Count")
)
df_all_zip["ZIPCODE"] = "All"

# Combine them
df_counts = pd.concat([df_counts, df_all_zip], ignore_index=True)

In [ ]:
# Sort by ZIPCODE, Overdose_Type, then time
df_counts = df_counts.sort_values(["ZIPCODE", "Overdose_Type", time_col])

# Convert the period to a timestamp for rolling
df_counts["PeriodStart"] = df_counts[time_col].dt.to_timestamp(how="start")

# Rolling 12-month average (for monthly data)
df_counts["Rolling12"] = df_counts.groupby(["ZIPCODE", "Overdose_Type"])[
    "Overdose_Count"
].transform(lambda x: x.rolling(window=12, min_periods=1).mean())

In [ ]:
df_counts["zipodtype"] = (
    df_counts["ZIPCODE"].astype(str) + "_" + df_counts["Overdose_Type"] + "_Count"
)

In [ ]:
df_counts.to_csv("time_series_with_zip_od_types_fix.csv", index=False)